<a href="https://colab.research.google.com/github/antongutiza/Fall2025/blob/main/Goal_2_Code_Generation_with_ReACT_Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Nova: ReACT Travel Planner (Colab / local)
# Paste this into Google Colab and run.
# Requires: pip install openai

import os
import json
import re
import getpass
from openai import OpenAI

# ---------- Setup OpenAI client ----------
# Try env var first; if not present, ask securely
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    api_key = getpass.getpass("Enter your OpenAI API key (sk-...): ").strip()

client = OpenAI(api_key=api_key)

# ---------- Helpers ----------
def extract_json(text):
    """Extract the first JSON object found in text and parse it."""
    # find first '{' and last '}' to try to parse JSON block
    try:
        start = text.index('{')
        end = text.rindex('}') + 1
        json_str = text[start:end]
        return json.loads(json_str)
    except Exception:
        # fallback: try to find triple-backtick block with json
        m = re.search(r"```(?:json)?\s*(\{.*?\})\s*```", text, re.S|re.M)
        if m:
            try:
                return json.loads(m.group(1))
            except Exception:
                return None
        return None

def ask_input(prompt, required=True, default=None):
    v = input(prompt).strip()
    if v == "" and default is not None:
        return default
    while required and v == "":
        v = input("Please provide a value — " + prompt).strip()
    return v

# ---------- Collect initial user preferences ----------
print("=== Nova: Travel Planning Assistant (ReACT) ===\nPlease provide the following preferences.\n")
destination = ask_input("1) What city or type of destination do you want to visit? (e.g., Barcelona or beach town): ")
budget = ask_input("2) What's your budget range? (e.g., low <$800, medium $800-1500, high $1500+): ")
interests = ask_input("3) What are your main interests? (comma-separated: food, nature, art, nightlife, etc.): ")
trip_style = ask_input("4) Do you prefer relaxation, adventure, or culture-focused activities?: ")

# Pack a preferences dict to pass into the assistant
preferences = {
    "destination": destination,
    "budget": budget,
    "interests": [i.strip() for i in interests.split(",") if i.strip()],
    "trip_style": trip_style
}

# ---------- System prompt instructing ReACT behavior and strict JSON output ----------
system_prompt = """
You are Nova, an intelligent travel planning assistant. Use the ReACT pattern: first Reason, then Act.

CONSTRAINTS:
- For every response, return a single valid JSON object (no extra commentary) with exactly these possible keys:
  - "Reason": a short string describing your analysis and what is missing or inferred.
  - "Act": an object that contains either:
      a) "Clarify": a list of specific short questions if you need more info to produce an itinerary, OR
      b) "Itinerary": a string with a clear 3-day day-by-day plan, plus
         optional fields "EstimatedCost" (string) and "Summary" (short string).
- If you require clarification, set "Act": {"Clarify": ["question1?", "question2?"]} and do NOT include "Itinerary".
- If no clarification is needed, set "Act": {"Itinerary": "...", "EstimatedCost": "...", "Summary": "..."}.
- Do NOT include other keys or markdown, and do NOT include any explanation outside the JSON object.
- Keep Reason concise (1-3 sentences). For Itinerary, provide day-by-day bullets or paragraphs.

Now, given the user preferences provided in the user message, follow the ReACT cycle and return the JSON described above.
"""

# ---------- Main loop: call assistant, handle clarifications ----------
max_rounds = 3
round_num = 0
context = {
    "preferences": preferences,
    "clarifications": {}
}

while round_num < max_rounds:
    round_num += 1

    # Build the user message including gathered preferences and any clarifications already provided
    user_message = {
        "role": "user",
        "content": (
            f"User preferences: {json.dumps(context['preferences'])}.\n"
            f"Previous clarifications (if any): {json.dumps(context['clarifications'])}\n\n"
            "Please follow the ReACT JSON format described in the system prompt to either (a) ask for specific clarifying questions "
            "if more details are needed, or (b) return a complete 3-day itinerary (Itinerary + EstimatedCost + Summary)."
        )
    }

    # Call the OpenAI chat completion
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # change if you don't have access
            messages=[
                {"role": "system", "content": system_prompt},
                user_message
            ],
            temperature=0.2,
            max_tokens=800
        )
    except Exception as e:
        print("API error:", e)
        break

    assistant_text = response.choices[0].message.content.strip()
    print("\n[raw assistant response]\n", assistant_text, "\n---\n")

    parsed = extract_json(assistant_text)
    if not parsed:
        print("Could not parse JSON from assistant response. Raw response printed above. Aborting.")
        break

    # Print Reason
    reason = parsed.get("Reason", "")
    print("Reason:", reason)

    act = parsed.get("Act", {})
    # If Act requests Clarify:
    clarify = act.get("Clarify")
    if clarify:
        print("\nAssistant requests clarification on:")
        for q in clarify:
            print(" -", q)
        # Ask user answers, store them as mapping
        for q in clarify:
            ans = ask_input(q + " ")
            # Use a short key based on question text
            key = "clarification_" + re.sub(r'[^a-z0-9]', '_', q.lower())[:40]
            context['clarifications'][key] = ans
        print("\nThanks — sending clarifications back to Nova and re-running reasoning...\n")
        continue  # loop to re-call the assistant with new clarifications

    # Otherwise, expect Itinerary
    itinerary = act.get("Itinerary")
    if itinerary:
        est = act.get("EstimatedCost", "N/A")
        summary = act.get("Summary", "")
        print("\n=== Generated 3-Day Itinerary ===\n")
        print(itinerary)
        print("\nEstimated cost:", est)
        if summary:
            print("\nSummary:", summary)
        print("\n=== End of itinerary ===")
        break

    # fallback
    print("Assistant returned an Act that is neither 'Clarify' nor 'Itinerary'. Raw Act:")
    print(json.dumps(act, indent=2))
    break

else:
    print("Reached maximum rounds without producing an itinerary.")


Enter your OpenAI API key (sk-...): ··········
=== Nova: Travel Planning Assistant (ReACT) ===
Please provide the following preferences.

1) What city or type of destination do you want to visit? (e.g., Barcelona or beach town): Philippines
2) What's your budget range? (e.g., low <$800, medium $800-1500, high $1500+): 1676
3) What are your main interests? (comma-separated: food, nature, art, nightlife, etc.): food
4) Do you prefer relaxation, adventure, or culture-focused activities?: relazation

[raw assistant response]
 {
  "Reason": "The user is interested in a relaxing food-focused trip to the Philippines but hasn't specified which islands or cities they prefer to visit.",
  "Act": {
    "Clarify": [
      "Which specific islands or cities in the Philippines are you interested in?",
      "What type of food experiences are you looking for (e.g., street food, fine dining, cooking classes)?"
    ]
  }
} 
---

Reason: The user is interested in a relaxing food-focused trip to the Phili

In [ ]:
pip install openai